In [1]:
from MiniTorch.tensor import Tensor, Parameter
from MiniTorch.functions import relu
from MiniTorch.optimizer import Optimizer
from MiniTorch.visualizer import trace, build_graph
import numpy as np

In [2]:
class Model:
    def __init__(self):
        self.layers = [
            Parameter(1, 1),
        ]

    def __call__(self, x):
        if not isinstance(x, Tensor):
            x = Tensor(x)
        
        out = x
        for layer in self.layers:
            out = layer(out)

        return out
    
    def parameters(self):
        return self.layers

In [14]:
def mse(x: Tensor, y):
    out = Tensor(0.5 * (x.data - y) ** 2, _children = (x,Tensor(y)), _op = "mse")
    
    def _backward():
        x.grad -= (y - x.data) * out.grad
        
    out._backward = _backward
    return out

In [4]:
nn = Model()

In [5]:
train_data = np.array(list(zip(
    np.arange(1, 100),
    2 * np.arange(1, 100)
)))

In [6]:
test_data = train_data

In [17]:
epochs = 1000
optimizer = Optimizer(nn.parameters(), learning_rate = 0.01)

In [30]:
old_loss = -1
for epoch in range(epochs):
    loss = 0
    for x, y in train_data:
        x, y = x / 100, y / 100
        out = nn([[x]])
        out = mse(out, y)
        
        optimizer.zero_grad()
        out.backward()
        optimizer.step()
        
        loss += 0.5 * (out.data - y) ** 2
    
    loss /= len(train_data)
    
    print(loss, old_loss)
    
    if loss == old_loss:
        break
    
    old_loss = loss
    
    print(f"{epoch = } | {loss = }")

[[0.66333333]] -1
epoch = 0 | loss = array([[0.66333333]])
[[0.66333333]] [[0.66333333]]


In [ ]:
train_acc = 0

for x, y in train_data:
    